# Example of dynamics learning of SIS on BA networks.

In [25]:
import numpy as np
import matplotlib.pyplot as plt

In this notebook, we show an example of dynamics learning of SIS dynamics on Barabasi-Albert networks. We first start by gathering the configuration of the experiment.

In [29]:
from dynalearn.config import ExperimentConfig

config = ExperimentConfig.default(
    "example-sis-ba", 
    "sis", 
    "ba", 
    path_to_data="./examples/data", 
    path_to_summary="./examples/summaries", 
    path_to_best="./examples/best", 
    seed=0
)
config.metrics.names = ("TrueLTPMetrics","GNNLTPMetrics","MLELTPMetrics")
config.train_details.num_samples = 1000
config.train_details.epochs = 10
config.train_details.use_groundtruth = 1


Then, we define the experiment from this configuration.


In [30]:
from dynalearn.experiments import Experiment
exp = Experiment(config, verbose=2)

Finally, we can run the experiment. We must perform certain tasks before the experiment is completed: 1) we generate the training and validation datasets, 2) we train the model using these datasets and finally 3) we compute the transition probabilities computed by the trained model.

In [ ]:
exp.run(["generate_data", "partition_val_dataset", "train_model", "compute_metrics"])

---Experiment example-sis-ba---
Current time: 2021-05-27 14:07:05

---Config---
name: example-sis-ba
path_to_data: ./examples/data/example-sis-ba
path_to_best: ./examples/best/example-sis-ba.pt
path_to_summary: ./examples/summaries
dynamics:
	name: SIS
	infection: 0.04
	recovery: 0.08
	init_param: None
	is_weighted: False
	is_multiplex: False
	lag: 1
	lagstep: 1

networks:
	name: BANetworkGenerator
	num_nodes: 1000
	m: 2

model:
	name: GNNSEDynamics
	gnn_name: DynamicsGATConv
	type: linear
	num_states: 2
	optimizer:
		name: RAdam
		lr: 0.001
		weight_decay: 0.0001
		betas: (0.9, 0.999)
		eps: 1e-08
		amsgrad: False

	in_activation: relu
	gnn_activation: relu
	out_activation: relu
	in_channels: [32, 32]
	gnn_channels: 32
	out_channels: [32, 32]
	heads: 2
	concat: True
	bias: True
	self_attention: True
	is_weighted: False
	is_multiplex: False
	network_layers: None
	in_size: 1
	out_size: 2
	out_act: softmax
	node_channels: [0]
	num_params: 8810
	lag: 1
	lagstep: 1

dataset:
	name: Discret

In [ ]:
import seaborn
seaborn.set_context("notebook")

def plot_ltp(experiment, ax):
    summary = experiment.metrics["TrueLTPMetrics"].data["summaries"]
    true_ltp = experiment.metrics["TrueLTPMetrics"].data["ltp"]
    gnn_ltp = experiment.metrics["GNNLTPMetrics"].data["ltp"]
    mle_ltp = experiment.metrics["MLELTPMetrics"].data["ltp"]
    agg = lambda ltp, in_s, out_s: LTPMetrics.aggregate(
            ltp, summary, 
            in_state=in_s, 
            out_state=out_s,
            axis=1, 
            reduce="mean", 
            err_reduce="percentile"
        )
    x_min, x_max = -np.inf, np.inf
    for i, (in_s, out_s) in enumerate(transitions):
        x, y, yl, yh = agg(true_ltp, in_s, out_s)
        ax.plot(
            x, y, color=colors["true"][i], linestyle=linestyles["true"][i],marker=markers["true"][i],linewidth=3
        )
        ax.fill_between(x, yl, yh, color=colors["true"][i], alpha=0.3)
        
        x, y, yl, yh = agg(gnn_ltp, in_s, out_s)
        ax.plot(
            x, y, color=colors["gnn"][i], linestyle=linestyles["gnn"][i],marker=markers["gnn"][i],linewidth=3
        )
        ax.fill_between(x, yl, yh, color=colors["gnn"][i], alpha=0.3)
        
        x, y, yl, yh = agg(mle_ltp, in_s, out_s)
        yerr = np.concatenate([np.expand_dims(y-yl,0), np.expand_dims(yh-y,0)], axis=0)
        ax.errorbar(
            x, 
            y, 
            yerr=yerr,
            color=colors["mle"][i], 
            linestyle=linestyles["mle"][i], 
            marker=markers["mle"][i], 
            alpha=0.3
        )
#         ax.plot(
#             x, y, color=colors["mle"][i], linestyle=linestyles["mle"][i], marker=markers["mle"][i], alpha=0.5
#         )
#         ax.fill_between(x, yl, yh, color=colors["mle"][i], alpha=0.3)
        
        if x.min() > x_min:
            x_min = x.min()
        if x.max() < x_max:
            x_max = x.max()
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([0, 1.1])
    return ax

fig, ax = plt.subplots(1, 1)

plot_ltp(exp, ax)